In [2]:
from scipy.io import arff
import pandas as pd
import numpy as np
import math

from numpy import unique
from numpy import where
from matplotlib import pyplot

data = arff.loadarff('./DefectData-master/DefectData/inst/extdata/terapromise/mccabe/PC3.arff')
df = pd.DataFrame(data[0])

In [3]:
df.shape

(1077, 38)

In [4]:
df.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,16.0,13.0,1.0,6.0,11.0,20.0,7.0,0.24,8.0,2.5,...,17.0,0.12,68.0,81.0,31.0,21.0,57.0,42.50,29.0,b'N'
1,2.0,7.0,0.0,0.0,7.0,10.0,4.0,0.29,4.0,2.5,...,9.0,0.17,9.0,22.0,5.0,8.0,24.0,33.33,14.0,b'Y'
2,1.0,13.0,5.0,0.0,0.0,22.0,7.0,0.37,10.0,2.2,...,23.0,0.33,38.0,53.0,19.0,19.0,21.0,0.00,19.0,b'N'
3,8.0,3.0,1.0,0.0,1.0,4.0,2.0,0.14,2.0,2.0,...,6.0,0.08,32.0,38.0,19.0,13.0,24.0,6.67,14.0,b'N'
4,1.0,5.0,2.0,1.0,1.0,6.0,3.0,0.15,2.0,3.0,...,9.0,0.13,27.0,33.0,19.0,14.0,23.0,9.52,20.0,b'N'


In [5]:
buggy = b'Y'
clean = b'N'
x = df.drop(['Defective'],axis = 'columns')

for i in range(1077):
    if df.iloc[i,37] == buggy:
        df.iloc[i,37] = 1
    else:
        df.iloc[i,37] = 0

y = df['Defective']
y = y.astype(int)
y.head()

0    0
1    1
2    0
3    0
4    0
Name: Defective, dtype: int32

In [6]:
buggy = 1
clean = 0

In [7]:
from sklearn.cluster import AffinityPropagation

af_model = AffinityPropagation(damping=0.9)
af_model.fit(x)
af_prediction = af_model.predict(x)

af_prediction

array([ 0,  8,  6, ..., 40, 41,  8], dtype=int64)

In [8]:
clusters = unique(af_prediction)
print(clusters)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41]


In [9]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if af_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if af_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


In [10]:
TP = 1
TN = 1
FP = 1
FN = 1

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == af_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient
MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.004150048681546136
Accuracy : 0.8695652173913043
F-Score : 0.013986013986013986


In [16]:
from sklearn.cluster import OPTICS

optics_model = OPTICS()
optics_prediction = optics_model.fit_predict(x)

optics_prediction[:5]

array([ 0, -1, 12, -1, 31])

In [17]:
clusters = unique(optics_prediction)
print(clusters)

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70
 71 72 73 74]


In [18]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if optics_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if optics_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [19]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == optics_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.06739274367350827
Accuracy : 0.49953574744661094
F-Score : 0.23546099290780137


In [20]:
from sklearn.cluster import MeanShift

MeanShift_model = MeanShift()
MeanShift_prediction = MeanShift_model.fit_predict(x)

clusters = unique(MeanShift_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if MeanShift_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if MeanShift_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 1
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == MeanShift_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.015420464376056271
Accuracy : 0.8719851576994434
F-Score : 0.014285714285714285


In [23]:
from sklearn.cluster import DBSCAN

DBSCAN_model = DBSCAN()
DBSCAN_prediction = DBSCAN_model.fit_predict(x)

clusters = unique(DBSCAN_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if DBSCAN_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if DBSCAN_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == DBSCAN_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1

TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.0114804404548961
Accuracy : 0.12523191094619665
F-Score : 0.2213047068538398


In [12]:
from pyclustering.cluster.xmeans import xmeans

xmeans_instance = xmeans(x)
xmeans_instance.process()
groups = xmeans_instance.get_clusters()

xmeans_prediction = [0]*len(y)

for i in range(len(groups)):
    for u in groups[i]:
        xmeans_prediction[u] = i

clusters = unique(xmeans_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if xmeans_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if xmeans_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == xmeans_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

ZeroDivisionError: float division by zero